In [ ]:
import os
import OpenDartReader
import FinanceDataReader as fdr

In [ ]:
stock_list = fdr.StockListing("KRX").dropna()

In [ ]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

In [ ]:
def extract_info_and_save(dart, path, year, name, code, report_type, report_type_code):
    report = dart.finstate(code, year, report_type_code)
    if report is None:
        pass
    else:
        submission_date = report['rcept_no'].iloc[0][:8]
        report = report[['fs_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'sj_nm']]
        report.rename({"fs_nm":"개별/연결",
                       "account_nm":"계정명",
                       "sj_nm":"재무제표명",
                       "thstrm_dt": "당기일자",
                       "thstrm_amount":"금액"}, axis = 1, inplace = True)
        report['금액'] = report['금액'].apply(convert_str_to_float)
        
        if name not in os.listdir(path):
            os.mkdir(path + "/" + name)
            
        report.to_csv(path + "/{}/{}_{}년_{}.csv".format(name, submission_date, year, report_type),
                      index = False, encoding = "euc-kr")

In [ ]:
import time
my_api = '6ac6c88791643c87a1e2e0b33040e9c953a8287d'
path = ''
dart = OpenDartReader(my_api)
for code, name in stock_list[['Symbol', 'Name']].values:
    for year in range(2016, 2022):
        for report_type, report_type_code in zip(["1분기보고서", "반기보고서", "3분기보고서", "사업보고서"],
                                                  ["11013", "11012", "11014", "11011"]):
            print(name, year, report_type)
            while True:
                try:
                    extract_info_and_save(dart = dart,
                                          path = r"qspy\data\재무제표",
                                          year = year,
                                          name = name,
                                          code = code,
                                          report_type = report_type,
                                          report_type_code = report_type_code)
                    time.sleep(0.5)
                    break
                    
                except:
                    time.sleep(60)